In [2]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
dating = pd.read_csv('./data/datingTestSet.txt',header=None,sep='\t')

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [18]:
train = dating[[0,1,2]]
target = dating[3]

In [20]:
from sklearn.preprocessing import Normalizer,StandardScaler,MinMaxScaler

In [23]:
# 因为喜欢程度是有关系的三种值，所以手动对其进行数字的映射
map_dic = {
    'largeDoses':0, 
    'smallDoses':1, 
    'didntLike':2
}
target = target.map(map_dic)
target

0      0
1      1
2      2
3      2
4      2
5      2
6      0
7      0
8      2
9      0
10     2
11     2
12     1
13     2
14     2
15     2
16     2
17     2
18     1
19     0
20     1
21     2
22     1
23     0
24     1
25     0
26     1
27     0
28     1
29     2
      ..
970    0
971    2
972    1
973    2
974    1
975    0
976    2
977    0
978    1
979    1
980    1
981    1
982    1
983    1
984    1
985    1
986    1
987    1
988    1
989    1
990    0
991    1
992    1
993    1
994    1
995    1
996    2
997    0
998    0
999    0
Name: 3, Length: 1000, dtype: int64

In [24]:
# 对特征进行无量纲化处理
train1 = Normalizer().fit_transform(train)
train2 = MinMaxScaler().fit_transform(train)
train3 = StandardScaler().fit_transform(train)

In [26]:
from sklearn.model_selection import train_test_split

# 比较不同的特征处理的结果
def split_predict_with_model(model,train,target,feature_project):
    X_train,X_test,y_train,y_test = train_test_split(train,target,test_size=0.2,random_state=1)
    score = model.fit(X_train,y_train).score(X_test,y_test)
    model_name = model.__class__.__name__
    print("{} 使用{}来处理特征的评分是 {}".format(model_name,feature_project,score))

In [27]:
knn = KNeighborsClassifier()
logistic = LogisticRegression()
tree = DecisionTreeClassifier()
gaussionNB = GaussianNB()
svc = SVC()

In [28]:
# 比较不同模型，不同特征处理得到的结果
def show_model_predict_results(model,train,target):
    train1 = Normalizer().fit_transform(train)
    train2 = MinMaxScaler().fit_transform(train)
    train3 = StandardScaler().fit_transform(train)
    trains = [train1,train2,train3]
    feature_project_names = ['Normalizer','MinMaxScaler','StandardScaler']
    for index,feature_train in enumerate(trains):
        split_predict_with_model(model,feature_train,target,feature_project_names[index])

In [29]:
show_model_predict_results(knn,train,target)

KNeighborsClassifier 使用Normalizer来处理特征的评分是 0.75
KNeighborsClassifier 使用MinMaxScaler来处理特征的评分是 0.935
KNeighborsClassifier 使用StandardScaler来处理特征的评分是 0.935


In [30]:
show_model_predict_results(logistic,train,target)

LogisticRegression 使用Normalizer来处理特征的评分是 0.305
LogisticRegression 使用MinMaxScaler来处理特征的评分是 0.885
LogisticRegression 使用StandardScaler来处理特征的评分是 0.91


In [31]:
show_model_predict_results(tree,train,target)

DecisionTreeClassifier 使用Normalizer来处理特征的评分是 0.7
DecisionTreeClassifier 使用MinMaxScaler来处理特征的评分是 0.96
DecisionTreeClassifier 使用StandardScaler来处理特征的评分是 0.96


In [32]:
show_model_predict_results(gaussionNB,train,target)

GaussianNB 使用Normalizer来处理特征的评分是 0.54
GaussianNB 使用MinMaxScaler来处理特征的评分是 0.935
GaussianNB 使用StandardScaler来处理特征的评分是 0.935


In [33]:
show_model_predict_results(svc,train,target)

SVC 使用Normalizer来处理特征的评分是 0.305
SVC 使用MinMaxScaler来处理特征的评分是 0.935
SVC 使用StandardScaler来处理特征的评分是 0.96


In [34]:
# 结论：使用StandardScaler对特征处理评分表现更高，算法可以选择决策树模型或者SVC模型

In [41]:
# 进一步对选择的算法进行参数调整
tree = DecisionTreeClassifier(max_depth=3)
show_model_predict_results(tree,train,target)

DecisionTreeClassifier 使用Normalizer来处理特征的评分是 0.81
DecisionTreeClassifier 使用MinMaxScaler来处理特征的评分是 0.96
DecisionTreeClassifier 使用StandardScaler来处理特征的评分是 0.96


In [51]:
svc = SVC(C=100)
show_model_predict_results(svc,train,target)

SVC 使用Normalizer来处理特征的评分是 0.305
SVC 使用MinMaxScaler来处理特征的评分是 0.965
SVC 使用StandardScaler来处理特征的评分是 0.965


In [ ]:
# 问题：算法评分有时候可能是假象，测试集上表现良好，但是在真是的预测环境下表现下滑
# 产生原因：对算法的训练和测试太单一，

In [117]:
def evalute_model(times,model,train,target):
    results = []
    for _ in range(times):
        # 为了让训练集和测试集的拆分更加随机，不再使用random_state参数，观察不同的拆分，得到的结果
        X_train,X_test,y_train,y_test = train_test_split(train,target,test_size=0.4,random_state=1)
        # 一个算法好坏，除了观察算法评分之外，还应该对算法的稳定性进行评估
        # 可以采取多次求解取方差（或者均值）
        result = model.fit(X_train,y_train).score(X_test,y_test)
        results.append(result)

    r = np.array(results)
    model_name = model.__class__.__name__
    
    print("{}模型，{}次预测，平均值是{}，方差是{}，最大值是{}，最小值是{}".format(model_name,times,r.mean(),r.std(),r.max(),r.min()))

In [118]:
evalute_model(50,svc,train3,target)

SVC模型，50次预测，平均值是0.9575，方差是0.0，最大值是0.9575，最小值是0.9575


In [119]:
evalute_model(50,tree,train3,target)

DecisionTreeClassifier模型，50次预测，平均值是0.9525，方差是0.0，最大值是0.9525，最小值是0.9525


In [110]:
evalute_model(50,svc,train3,target)

SVC模型，50次预测，平均值是0.9666666666666667，方差是0.0，最大值是0.9666666666666667，最小值是0.9666666666666667


In [113]:
evalute_model(50,tree,train3,target)

DecisionTreeClassifier模型，50次预测，平均值是0.96，方差是0.0，最大值是0.96，最小值是0.96


In [120]:
# 使用机器学习的交叉验证来解决样本集不均衡导致的偶然性问题
from sklearn.model_selection import cross_val_score,cross_val_predict
# KFold是用于拆分数据集的一个类
from sklearn.model_selection import KFold

In [132]:
# 使用交叉验证来测量不同的算法评分指标
def cross_score_result(model,train,target):
    # n_splits 数据集分类的份数
    kfold = KFold(n_splits=10,shuffle=True)

    
    # 参数一：要进行交叉验证的算法模型
    # 参数二、三：要进行拆分的数据特征集合和标签集合
    # 参数三:kfold对象，描述了数据集如何进行拆分
    # 返回值：多次预测的算法评分
    scores = cross_val_score(model,train,target,cv=kfold)
    model_name = model.__class__.__name__
    print("{}模型 方差:{},均值:{},最大值:{},最小值{}".format(model_name,scores.std(),scores.mean(),scores.max(),scores.min()))

In [133]:
cross_score_result(svc,train3,target)

SVC模型 方差:0.019209372712298528,均值:0.9490000000000001,最大值:0.98,最小值0.91


In [134]:
cross_score_result(tree,train3,target)

DecisionTreeClassifier模型 方差:0.016881943016134125,均值:0.9550000000000001,最大值:0.98,最小值0.92


In [137]:
cross_score_result(knn,train3,target)

KNeighborsClassifier模型 方差:0.019078784028338885,均值:0.946,最大值:0.98,最小值0.93


In [138]:
cross_score_result(logistic,train3,target)

LogisticRegression模型 方差:0.02879236009777594,均值:0.909,最大值:0.94,最小值0.84


In [139]:
cross_score_result(gaussionNB,train3,target)

GaussianNB模型 方差:0.03041381265149108,均值:0.9350000000000002,最大值:0.98,最小值0.87


In [140]:
cross_score_result(svc,train2,target)

SVC模型 方差:0.017320508075688763,均值:0.9600000000000002,最大值:0.98,最小值0.92


In [141]:
cross_score_result(svc,train1,target)

SVC模型 方差:0.03935733730830885,均值:0.329,最大值:0.38,最小值0.26


In [142]:
cross_score_result(tree,train2,target)

DecisionTreeClassifier模型 方差:0.026381811916545813,均值:0.9480000000000001,最大值:0.98,最小值0.91
